In [6]:
sys.path

['C:\\Python310\\python310.zip',
 'C:\\Python310\\DLLs',
 'C:\\Python310\\lib',
 'C:\\Python310',
 'c:\\Users\\codnw\\PycharmProjects\\linear_mixture_mdp\\LSVI-DC\\.venv',
 '',
 'c:\\Users\\codnw\\PycharmProjects\\linear_mixture_mdp\\LSVI-DC\\.venv\\lib\\site-packages',
 'c:\\Users\\codnw\\PycharmProjects\\linear_mixture_mdp\\LSVI-DC\\.venv\\lib\\site-packages\\win32',
 'c:\\Users\\codnw\\PycharmProjects\\linear_mixture_mdp\\LSVI-DC\\.venv\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\codnw\\PycharmProjects\\linear_mixture_mdp\\LSVI-DC\\.venv\\lib\\site-packages\\Pythonwin']

In [8]:
import os, sys
# sys.path.append('../')
from tqdm import tqdm
import random
from env.env import *
from algorithms.ucrl2_vtr import *
from algorithms.optimal_policy import *

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
runs = 1
seeds = [123*(i+1) for i in range(runs)]

In [ ]:
T = 10000  # 3000 이후로 infeasible 문제
nState = 6
delta = 0.01
lam = 1
epsilon = min(0.01, 1/ np.sqrt(T))
# epsilon = 1/np.sqrt(T)
env = make_riverSwim(T=T, nState=nState)
# c_ = [1, 5e-1, 1e-1, 5e-2, 1e-2]

run_returns = []

for run in tqdm(range(runs)):
    
    random.seed(seeds[run])
        
    agent = UCRL2_VTR(env, T=T, c=1e-2, delta=0.01, lam=1, epsilon=epsilon)    
    episodic_return = agent.run()
    run_returns.append(episodic_return)
    resultDir = '../data/riverSwim/S=' + str(nState)+ ', T=' + str(10000) +'/UCRL2-VTR'
    if not os.path.exists(resultDir):
        os.makedirs(resultDir)  # Create the directory if it doesn't exist
    np.save(resultDir + '/return' + str(run)+'.npy', episodic_return)


In [1]:
import os
import sys
sys.path.append('../')
import random
import numpy as np
from tqdm import tqdm
from env.env import *
from algorithms.ucrl2_vtr import UCRL2_VTR
import multiprocessing as mp
import matplotlib.pyplot as plt
import seaborn as sns

# Function to run a single experiment
def run_experiment(run, seed, T, nState, delta, lam, epsilon, resultDir):
    random.seed(seed)
    
    # Create the environment
    env = make_riverSwim(T=T, nState=nState)
    
    # Initialize the agent
    agent = UCRL2_VTR(env, T=T, c=1e-2, delta=delta, lam=lam, epsilon=epsilon)
    # Run the agent
    episodic_return = agent.run()
    print("seed %d done"%(seed))

    # Save the result
    if not os.path.exists(resultDir):
        os.makedirs(resultDir)  # Create the directory if it doesn't exist
    np.save(resultDir + f'/return{run}.npy', episodic_return)

    return episodic_return

def main():
    runs = 4  # Adjust this based on the number of parallel runs
    seeds = [123 * (i + 1) for i in range(runs)]
    T = 100
    nState = 6
    delta = 0.01
    lam = 1
    epsilon = min(0.01, 1 / np.sqrt(T))

    resultDir = '../data/riverSwim/S=' + str(nState) + ', T=' + str(T) + '/UCRL2-VTR'
    
    # Use multiprocessing to run experiments in parallel
    pool = mp.Pool(mp.cpu_count())  # Use all available CPUs

    # Use pool.starmap to distribute the runs in parallel
    results = pool.starmap(run_experiment, [(run, seeds[run], T, nState, delta, lam, epsilon, resultDir) for run in range(runs)])

    pool.close()  # Close the pool to prevent new tasks from being submitted
    pool.join()  # Wait for all worker processes to finish

    episodes = np.arange(T)

    plt.figure()
    data_mean = np.mean(results, axis=0)
    data_std = np.std(results, axis=0)
        
    plt.fill_between(episodes, data_mean + data_std, data_mean - data_std, alpha=0.2)
    plt.plot(episodes, data_mean, linewidth=1.8)
    plt.title("RiverSwim: nState=6, T=100")
    plt.xlabel("The Number of Episodes")

    plt.ylabel("Episodic Returns")
    plt.show()

    return results

if __name__ == '__main__':
    run_returns = main()
    print("All experiments completed!")


NameError: name 'make_riverSwim' is not defined

In [ ]:
episodes = np.arange(T)

plt.figure()
data_mean = np.mean(run_returns, axis=0)
data_std = np.std(run_returns, axis=0)
    
plt.fill_between(episodes, data_mean + data_std, data_mean - data_std, alpha=0.2)
plt.plot(episodes, data_mean, linewidth=1.8)
plt.title("RiverSwim: nState=4, epLen=15")
plt.xlabel("The Number of Episodes")

plt.ylabel("Episodic Returns")
plt.show()